In [1]:
#train with seperate encoder and predictor models bc I can't get transfer learning to work
import math, random, sys
sys.path.insert(0, '/home/marcase/hgraph2graph/')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import networkx as nx
import rdkit
import numpy as np
import argparse
import os
from tqdm.auto import tqdm
import pickle
from hgraph import *
from hgraph.inc_graph import *
from hgraph.encoder import *
import matplotlib.pyplot as plt
from hgraph.predict import HierPredict

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = '/home/marcase/hgraph2graph/data/chembl/cyclic_peptide_vocab.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)
vocab.vocab[21][0]
vocab.hvocab

['C',
 'C1=CC=CC=C1',
 'C1=CNC=C1',
 'C1=CNC=N1',
 'C1CCNC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCSCCCSCCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCNCCSCCCSCCNCCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCSCCCSCCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCSCCCSCCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCNCCSCCCSCCNCCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCSCCCSCCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCSCCCSCCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCSCCCSCCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCNCCSCCCSCCNCCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCCNCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCCNCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCCNCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCCNCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCNCCCSCCCSC1',
 'C1CNCCNCCNCCNCCSCCCSCCNCCN

In [4]:
class Args:
    train = '/home/marcase/hgraph2graph/predict/preprocessed_master/preprocessed_train/'
    train_labels = '/home/marcase/hgraph2graph/predict/preprocessed_master/preprocessed_train_labels/'
    test = '/home/marcase/hgraph2graph/predict/preprocessed_master/preprocessed_test/'
    test_labels = '/home/marcase/hgraph2graph/predict/preprocessed_master/preprocessed_test_labels/'
    vocab = vocab
    save_dir = 'test/'
    atom_vocab = common_atom_vocab
    load_model = None
    seed = 7
    rnn_type = 'LSTM'
    hidden_size=125
    embed_size=250
    batch_size=8
    latent_size=32
    depthT=15
    depthG=15
    diterT=1
    diterG=3
    dropout=0.2
    lr = 5e-4
    clip_norm=5.0
    step_beta=0.001
    max_beta=1.0
    warmup=10000
    kl_anneal_iter=2000
    epoch=2000
    anneal_rate=0.9
    anneal_iter=25000
    print_iter=50
    save_iter=1000000
    model = '/home/marcase/hgraph2graph/ckpt/cyclic_peptide_pretrained/model.ckpt.110000'
    load_model = True
    nsample = 1
    label_size = 2
    run_test = True
    separate_predict = False
    
args=Args()

#get model
if args.separate_predict:
    encoder = HierVAE(args).cuda()
    model = HierPredict(args).cuda()
else:
    model = HierVAE(args).cuda()

In [5]:
torch.manual_seed(args.seed)
random.seed(args.seed)

#initialize weights
if args.separate_predict:
    for param in encoder.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)
    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)   
else:
    for param in model.parameters():
        if param.dim() == 1:
            nn.init.constant_(param, 0)
        else:
            nn.init.xavier_normal_(param)        

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)
# scheduler = lr_scheduler.ExponentialLR(optimizer, args.anneal_rate)

#load model if args.load_model == True
if args.load_model:
    print('continuing from checkpoint ' + args.model)
    model_state, optimizer_state, total_step, beta = torch.load(args.model)
    
    if args.separate_predict:
        encoder.load_state_dict(model_state)
    else:
        #initialize weights in model.predict that don't exist from pre-training
        for key in model.predict.state_dict().keys():
            model_state['predict.' + key] = model.predict.state_dict()[key]
        
        model.load_state_dict(model_state)
else:
    total_step = beta = 0

param_norm = lambda m: math.sqrt(sum([p.norm().item() ** 2 for p in m.parameters()]))
grad_norm = lambda m: math.sqrt(sum([p.grad.norm().item() ** 2 for p in m.parameters() if p.grad is not None]))




continuing from checkpoint /home/marcase/hgraph2graph/ckpt/cyclic_peptide_pretrained/model.ckpt.110000


In [6]:
meters = np.array([])
meters_list = list(meters)
validation_list = list()
total_step = 0

In [7]:
#main train loop
for epoch in range(args.epoch):
    random.seed(args.seed)
    dataset_x = DataFolder(args.train, args.batch_size,shuffle = False)
    dataset_y = DataFolder(args.train_labels, args.batch_size,shuffle = False)
    dataset_x.data_files = ['tensors-'+str(i)+'.pkl' for i in range(len(dataset_x.data_files))]
    dataset_y.data_files = ['tensors_labels-'+str(i)+'.pkl' for i in range(len(dataset_y.data_files))]
    model.train()
    for batch_x,batch_y in zip(dataset_x,dataset_y):
        total_step += 1
        model.zero_grad()
        y_pred = model(*batch_x, beta=beta,decode=False,predict=True) 
        y_true = torch.Tensor([int(y) for y in batch_y]).cuda()
        y_true = y_true.type(torch.LongTensor).cuda()
        loss = criterion(y_pred,y_true)
        accuracy = torch.sum(torch.argmax(y_pred, dim=1).cuda() == y_true)/len(y_true)
        # loss = Variable(loss, requires_grad = True)
        loss.backward()
        optimizer.step()

        meters = np.array([loss.item(),accuracy.cpu()])
        meters_list.append(meters)

        if total_step % args.print_iter == 0:
            print("[%d] Beta: %.3f, loss: %.3f, accuracy: %.3f, PNorm: %.2f, GNorm: %.2f" % (total_step, beta, meters[0], meters[1], param_norm(model), grad_norm(model)))
            sys.stdout.flush()
            meters *= 0
        
        if total_step % args.save_iter == 0:
            ckpt = (model.state_dict(), optimizer.state_dict(), total_step, beta)
            torch.save(ckpt, os.path.join(args.save_dir, f"model.ckpt.{total_step}"))

        # if total_step % args.anneal_iter == 0:
        #     scheduler.step()
        #     print("learning rate: %.6f" % scheduler.get_lr()[0])

        if total_step >= args.warmup and total_step % args.kl_anneal_iter == 0:
            beta = min(args.max_beta, beta + args.step_beta)
    
    #"validation" set
    if args.run_test:
        model.eval()
        dataset_x = DataFolder(args.test, args.batch_size,shuffle = False)
        dataset_y = DataFolder(args.test_labels, args.batch_size,shuffle = False)
        dataset_x.data_files = ['tensors-'+str(i)+'.pkl' for i in range(len(dataset_x.data_files))]
        dataset_y.data_files = ['tensors_labels-'+str(i)+'.pkl' for i in range(len(dataset_y.data_files))]
        random.seed()
        i=0
        accuracy_list = list()
        for batch_x,batch_y in zip(dataset_x,dataset_y):
            batch_x0 = batch_x
            batch_y0 = batch_y
            y_pred = model(*batch_x, beta=beta,decode=False,predict=True) 
            y_pred = torch.argmax(y_pred, dim=1)
            y_true = torch.Tensor([int(y) for y in batch_y0]).cuda()
            y_true = y_true.type(torch.LongTensor).cuda()
            accuracy_list.append((torch.sum(y_pred == y_true)/len(y_pred)).item())
            i += 1


        print('Accuracy on validation set: %.3f' % np.average(accuracy_list))
        validation_list.append((torch.sum(y_pred == y_true)/len(y_pred)).item())

[50] Beta: 0.050, loss: 0.717, accuracy: 0.500, PNorm: 357.67, GNorm: 1.03


KeyboardInterrupt: 